In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


Read the Wine dataset.

In [ ]:
df = pd.read_csv('wine.csv')

df

Extract our features.

In [ ]:
df_features = df.loc[:, 'Alcohol':]

df_features

In [ ]:
df_label = df[['Cultivar']]

df_label

In [ ]:
df_label.values

Split the data for training and testing.

In [ ]:

y = df.iloc[:,0].values

x_train, x_test, y_train, y_test = train_test_split(
	df_features.values, df_label.values, test_size=0.1, stratify=y)


Appy standardization to our features.

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


Appy PCA for dimension reduction.

In [ ]:
pca = PCA(n_components=5)
pca.fit(x_train)

print('explained variance: ', pca.explained_variance_ratio_)
print('explained variance: ', pca.explained_variance_ratio_.sum())

Transform both our Training and Test data using the fitted PCA model.

In [ ]:
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


Re-label our labels to start from 0 (required by to_categorical() function).

In [ ]:
le = LabelEncoder()
le.fit(y)

le.classes_

Print our labels into encoded categorical numbers.

In [ ]:
y_train_le = le.transform(y_train)
y_test_le = le.transform(y_test)

print(y_train_le)
print(y_test_le)

Perform One-Hot Encoding on our categorical numbers.

In [ ]:
y_train_1hot = tf.keras.utils.to_categorical(y_train_le, 3)
y_test_1hot = tf.keras.utils.to_categorical(y_test_le, 3)

print(y_train_1hot)
print(y_test_1hot)

Create the model.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, input_shape=(x_train.shape[1],), activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Training the model.

In [ ]:
history = model.fit(x_train, y_train_1hot, epochs=300)

Plot our "loss" and "accuracy" graphs.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

ax[0].plot(history.history['loss'])
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_title('Loss Curve')

ax[1].plot(history.history['acc'])
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Accuracy Curve')

plt.show()

Perform auto evaluation.

In [ ]:
loss, accuracy = model.evaluate(x=x_test, y=y_test_1hot)

print('loss = ', loss)
print('accuracy = ', accuracy)

Eye-ball predicted values vs actual values.

In [ ]:
predictions = model.predict(x=x_test)
for i in np.arange(len(predictions)):
	print('Actual: ', y_test_1hot[i], 'Predicted: ', predictions[i])

Perform manual calculation for accuracy in predictions.

In [ ]:
n_preds = len(predictions)

correct = 0
wrong = 0

for i in np.arange(n_preds):
    pred_max = np.argmax(predictions[i])
    actual_max = np.argmax(y_test_1hot[i])
    if pred_max == actual_max:
        correct += 1
    else:
        wrong += 1

print('correct: {0}, wrong: {1}'.format(correct, wrong))
print('accuracy =', correct/n_preds)
